In [1]:
#install packages
!pip install transformers
#import packages
import pandas as pd
import matplotlib.pyplot as plt
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification

from sklearn.metrics import f1_score

from sklearn.model_selection import train_test_split

import numpy as np
import random


#Read data from git:
#https://raw.githubusercontent.com/FANMISUA/TweetAENormalization/main/ADENormalization/Data/CADEC/3.csv
# URL of the CSV file
csv_url = "https://raw.githubusercontent.com/FANMISUA/TweetAENormalization/main/ADENormalization/Data/CADEC/3.csv"

# Read the CSV file into a pandas DataFrame
column_names = ["TT", "llt_code", "ade", "soc_code"]
cadec_all = pd.read_csv(csv_url,names=column_names, header=None)

# Display the first few rows of the DataFrame
print(cadec_all.shape)

# Remove duplicate rows based on the 'ade' column
cadec_unique = cadec_all.drop_duplicates(subset='ade')

# Display the resulting DataFrame
print(cadec_unique.shape)
# Count occurrences of each 'soc_code'
soc_code_counts = cadec_unique['soc_code'].value_counts()
# Sort the counts from high to low and print the result
print(soc_code_counts)

#get top 3 of the SMM4H list
#['10018065','10037175','10029205','10022891','10028395','10017947']
top3SMM4H = [10018065,10037175,10029205]
top6SMM4H = [10018065,10037175,10029205,10022891,10028395,10017947]

# Filter DataFrame
filtered_data3 = cadec_unique[cadec_unique['soc_code'].isin(top3SMM4H)]
filtered_data6 = cadec_unique[cadec_unique['soc_code'].isin(top6SMM4H)]

# Select only the Term and SOC columns
CADECtop3inSMM4H = filtered_data3[['ade', 'soc_code']]
CADECtop6inSMM4H = filtered_data6[['ade', 'soc_code']]

print(CADECtop3inSMM4H)
data = CADECtop3inSMM4H
# Convert to DataFrame
df = pd.DataFrame(CADECtop3inSMM4H)

import pandas as pd

# Get unique labels and their counts
label_counts = df['soc_code'].value_counts()

# Sort labels by counts
sorted_labels = label_counts.index.tolist()

# Create label dictionary with labels assigned from 1 to the number of unique labels
label_dict = {label: i for i, label in enumerate(sorted_labels, 0)}

# Print label dictionary
print(label_dict)

label_df = pd.DataFrame.from_dict(label_dict, orient='index', columns=['Label'])

# Reset index to make the index a regular column
label_df.reset_index(inplace=True)

# Rename columns
label_df.columns = ['SOC Code', 'Label']

# Sort DataFrame by label
label_df = label_df.sort_values(by='Label')

# Print the DataFrame
print(label_df)

df['label'] = df.soc_code.replace(label_dict)

def custom_train_test_split(X, y, test_size=0.15, random_state=None):
    classes, counts = np.unique(y, return_counts=True)
    min_class_count = min(counts)

    # Find classes with only one or two instances
    single_or_double_instance_classes = classes[np.logical_or(counts == 1, counts == 2)]

    # Remove instances of single-instance or two-instance classes
    X_filtered = X[~np.isin(y, single_or_double_instance_classes)]
    y_filtered = y[~np.isin(y, single_or_double_instance_classes)]

    if len(y_filtered) < 2:
        raise ValueError("No classes have more than two instances after filtering.")

    # Perform stratified split on the filtered dataset
    X_train, X_val, y_train, y_val = train_test_split(X_filtered, y_filtered, test_size=test_size, random_state=random_state, stratify=y_filtered)

    # Randomly assign instances of single-instance classes to training or testing sets
    for class_label in single_or_double_instance_classes:
        class_indices = np.where(y == class_label)[0]
        np.random.shuffle(class_indices)

        if len(class_indices) <= 2:
            # Only one instance, randomly assign to training or testing set
            if np.random.rand() < test_size:
                X_val = np.concatenate((X_val, X[class_indices]))
                y_val = np.concatenate((y_val, y[class_indices]))
            else:
                X_train = np.concatenate((X_train, X[class_indices]))
                y_train = np.concatenate((y_train, y[class_indices]))

    return X_train, X_val, y_train, y_val


#evaluation
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}

    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')


(5962, 4)
(3348, 4)
soc_code
10028395    962
10018065    654
10037175    401
10017947    300
10029205    286
10040785    184
10007541     92
10038738     91
10022891     82
10015919     67
10038604     59
10038359     50
10022117     35
10047065     25
10013993     16
10019805     15
10041244      7
10027433      6
10021881      5
10021428      4
10014698      3
10005329      3
10029104      1
Name: count, dtype: int64
                            ade  soc_code
926            voracious hunger  10018065
927            loss of appetite  10018065
929            lack of appetite  10018065
931                    anorexia  10018065
932                    anorexic  10018065
...                         ...       ...
5326  short term memory lacking  10037175
5328      couldn't eat or drink  10037175
5329              Could not eat  10037175
5331           can't eat normal  10037175
5332   Disturbed sleep patterns  10037175

[1341 rows x 2 columns]
{10018065: 0, 10037175: 1, 10029205: 2}
   SOC C

In [ ]:
# Define the random seeds and other parameters
seed_values = list(range(2, 42, 2))
batch_size = 8
epochs = 10

# Placeholder for accuracies
accuracies = []
all_accuracies = {label: [] for label in range(len(label_dict))}


# Loop over each seed value
for seed_val in seed_values:
  # Set seeds
  random.seed(seed_val)
  np.random.seed(seed_val)
  torch.manual_seed(seed_val)
  torch.cuda.manual_seed_all(seed_val)

  X_train, X_val, y_train, y_val = custom_train_test_split(df.index.values, df.label.values, test_size=0.2, random_state=42)
  df['data_type'] = ['not_set']*df.shape[0]
  df.loc[X_train, 'data_type'] = 'train'
  df.loc[X_val, 'data_type'] = 'val'
  print(df.groupby(['soc_code', 'label', 'data_type']).count())

  result = df.groupby(['soc_code', 'label', 'data_type']).size().reset_index(name='count')
  result_sorted = result.sort_values(by='label')

  # Group by soc_code, label, and data_type
  grouped = df.groupby(['soc_code', 'label', 'data_type']).size().reset_index(name='count')
  # Filter data for training and validation
  train_counts = grouped[grouped['data_type'] == 'train']
  val_counts = grouped[grouped['data_type'] == 'val']

  tokenizer = BertTokenizer.from_pretrained('distilbert-base-uncased',
                                            do_lower_case=True)

  encoded_data_train = tokenizer.batch_encode_plus(
      df[df.data_type=='train'].ade.values,
      add_special_tokens=True,
      return_attention_mask=True,
      pad_to_max_length=True,
      max_length=256,
      return_tensors='pt'
  )

  encoded_data_val = tokenizer.batch_encode_plus(
      df[df.data_type=='val'].ade.values,
      add_special_tokens=True,
      return_attention_mask=True,
      pad_to_max_length=True,
      max_length=256,
      return_tensors='pt'
  )

  input_ids_train = encoded_data_train['input_ids']
  attention_masks_train = encoded_data_train['attention_mask']
  labels_train = torch.tensor(df[df.data_type=='train'].label.values)

  input_ids_val = encoded_data_val['input_ids']
  attention_masks_val = encoded_data_val['attention_mask']
  labels_val = torch.tensor(df[df.data_type=='val'].label.values)

  dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
  dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

  model = BertForSequenceClassification.from_pretrained("distilbert-base-uncased",
                                                        num_labels=len(label_dict),
                                                        output_attentions=False,
                                                        output_hidden_states=False)

  #data loader
  from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

  dataloader_train = DataLoader(dataset_train,
                                sampler=RandomSampler(dataset_train),
                                batch_size=batch_size)

  dataloader_validation = DataLoader(dataset_val,
                                    sampler=SequentialSampler(dataset_val),
                                    batch_size=batch_size)

  from transformers import AdamW, get_linear_schedule_with_warmup

  #training parameters
  optimizer = AdamW(model.parameters(),
                    lr=1e-5,
                    eps=1e-8)


  scheduler = get_linear_schedule_with_warmup(optimizer,
                                              num_warmup_steps=0,
                                              num_training_steps=len(dataloader_train)*epochs)

  # Assuming device is cuda
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  # Move model to GPU
  model.to(device)
  print(f"Device used: {device}")


  #training
  import random

  def evaluate(dataloader_val):

      model.eval()

      loss_val_total = 0
      predictions, true_vals = [], []

      for batch in dataloader_val:

          batch = tuple(b.to(device) for b in batch)

          inputs = {'input_ids':      batch[0],
                    'attention_mask': batch[1],
                    'labels':         batch[2],
                  }

          with torch.no_grad():
              outputs = model(**inputs)

          loss = outputs[0]
          logits = outputs[1]
          loss_val_total += loss.item()

          logits = logits.detach().cpu().numpy()
          label_ids = inputs['labels'].cpu().numpy()
          predictions.append(logits)
          true_vals.append(label_ids)

      loss_val_avg = loss_val_total/len(dataloader_val)

      predictions = np.concatenate(predictions, axis=0)
      true_vals = np.concatenate(true_vals, axis=0)

      return loss_val_avg, predictions, true_vals

  for epoch in tqdm(range(1, epochs+1)):

      model.train()

      loss_train_total = 0

      progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
      for batch in progress_bar:

          model.zero_grad()

          batch = tuple(b.to(device) for b in batch)

          inputs = {'input_ids':      batch[0],
                    'attention_mask': batch[1],
                    'labels':         batch[2],
                  }

          outputs = model(**inputs)
          # outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)


          loss = outputs[0]
          loss_train_total += loss.item()
          loss.backward()

          torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

          optimizer.step()
          scheduler.step()

          progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})


      torch.save(model.state_dict(), f'./finetuned_BERT_epoch_{epoch}.model')

      tqdm.write(f'\nEpoch {epoch}')

      loss_train_avg = loss_train_total/len(dataloader_train)
      tqdm.write(f'Training loss: {loss_train_avg}')

      val_loss, predictions, true_vals = evaluate(dataloader_validation)
      val_f1 = f1_score_func(predictions, true_vals)
      tqdm.write(f'Validation loss: {val_loss}')
      tqdm.write(f'F1 Score (Weighted): {val_f1}')

  _, predictions, true_vals = evaluate(dataloader_validation)
  accuracy_per_class(predictions, true_vals)

  #plot evaluation
  import matplotlib.pyplot as plt

  def accuracy_per_class(predictions, true_vals):
      pred_flat = np.argmax(predictions, axis=1).flatten()
      labels_flat = true_vals.flatten()
      accuracy_dict = {}
      count_dict = {}
      for label in np.unique(labels_flat):
          y_preds = pred_flat[labels_flat == label]
          y_true = labels_flat[labels_flat == label]
          accuracy_dict[label] = np.sum(y_preds == y_true) / len(y_true)
          count_dict[label] = len(y_true)
      return accuracy_dict, count_dict

  accuracy_dict, count_dict = accuracy_per_class(predictions, true_vals)
  for label, accuracy in accuracy_dict.items():
      all_accuracies[label].append(accuracy)
  print(seed_val, accuracy_dict, count_dict )

print(all_accuracies)
# Compute average and standard deviation of accuracy
avg_accuracy = {label: np.mean(accs) for label, accs in all_accuracies.items()}
std_accuracy = {label: np.std(accs) for label, accs in all_accuracies.items()}


                          ade
soc_code label data_type     
10018065 0     train      523
               val        131
10029205 2     train      229
               val         57
10037175 1     train      320
               val         81


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Epoch 3:   0%|          | 0/134 [00:00<?, ?it/s]


Epoch 3
Training loss: 0.44247604192081674
Validation loss: 0.648552839342943
F1 Score (Weighted): 0.7751849243282896


Epoch 4:   0%|          | 0/134 [00:00<?, ?it/s]


Epoch 4
Training loss: 0.33486557202830686
Validation loss: 0.6071913326843915
F1 Score (Weighted): 0.8197848268212026


Epoch 5:   0%|          | 0/134 [00:00<?, ?it/s]